Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos um valor práximo de zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



Importação da função 'random' e reutilização de funções criadas no arquivo 'funcoes.py', pois ainda são úteis para a resolução deste experimento.

In [5]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



Precisamos encontrar uma solução para que uma mochila - _que aguenta até 15 unidades de massa_ - guarde a maior quantidade de produtos, e, que ainda tenha o valor monetário mais alto possível, ou seja, os produtos mais caros da lista (temos um problema de maximização).

Contudo, não podemos nos esquecer do **peso** (nosso limitador). Sendo assim, devemos lembrar que nem sempre o maior conjunto de produtos que tem o valor mais alto, se encaixa necessariamente em até 15 (Unidades de Massa). Não devemos ultrapassar o peso, caso contrário: _"mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica"_ 
**:(**

A intenção é utilizar uma lógica já contruída no problema das caixas binárias. Nossa mochila será representada pelos produtos escolhidos a partir deste dicionário.

Para começar, definimos constantes relacionadas à busca, e ao problema em questão.  Se alteradas, as funções mudam a problemática e afetam a eficiência do algoritmo.


In [6]:
### CONSTANTES

# relacionadas à busca, apropriadas do nosso problema das caixas binárias
TAMANHO_POP = 20 # tamanho da população
NUM_GERACOES = 100 # número total de gerações
CHANCE_CRUZAMENTO = 0.5 # chance de indivíduos geradores
CHANCE_MUTACAO = 0.05 # chance de genes mutados

# relacionadas ao problema a ser resolvido
# se alteradas, afetam totalmente o experimento, já que estamos tratando do nosso fator limitante.
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys())) # nomes do objetos ficarão em ordem alfabética

Na definição das constantes, tem-se: 
- _"TAMANHO_POP"_, relacionada ao tamanho da população, composta por 20 indivíduos em todas as gerações. 
- _"NUM_GERACOES"_,  o número total de gerações a ser criado é 100.
- _"CHANCE_CRUZAMENTO"_ e _"CHANCE_MUTACAO"_, representam indivíduos em uma população que nem sempre cruzam para gerar descendentes e genes que nem sempre sofrem mutação. 

Para constantes relacionadas a problemática, tem-se:
- _"OBJETOS"_ que podem ser selecionados para o conjunto na mochila 
- _"LIMITE_DE_PESO"_ Peso máximo (15 Unidades de Massa).

Na próxima etapa, definimos funções locais, que utilizam de valores padrões já informados e utilizados apenas neste notebook.

In [7]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [8]:
# aqui está toda a lógica para a resolução do problema

# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf") # qualquer valor maior que menos infinito (é problema de maximização)
melhor_individuo_ja_visto = [0] * NUM_OBJETOS # garante o melhor indivíduo na posição 0, com genes definidos


for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy() # utilizamos copy() para não ter o perigo de alterar o elemento original na população.
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO: #entra no hall da fama, quem cumpre ambos os requisitos
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:") # atenção para 'if pega_ou_não', definimos 1 para pegar e 0 para não pegar.
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 16400 | Peso: 13.6

Você deve pegar os seguintes itens:
+ Harry Potter: ele voltou, confia!
+ Quadrinho super raro do Aranha-Homem da vida real
+ docinhos para o stress
+ tablet
+ vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 16400 dinheiros e peso de 13.6 unidades de massa.


## Conclusão



O objetivo era, selecionar a maior quantidade de itens que agregam mais valor monetário em uma mochila, sem que esta, excedesse o peso máximo de 15 Unidades de massa. 

Recapitulando os processos feitos para a resolução deste exercício, e lembrando de algoritmos genéticos, tem-se:
- Indivíduos são os itens do dicionário.
- Fitness calculado, uma vez que é necessário somar valores do itens e verificar o peso da mochila, em todas as populações.
- Encontrando o maior valor da lista de fitness, armazena-se a posição correspondente.
- Armazenando o respectivo indivíduo como cópia, para não ocorrer risco de modificação na lista original da população.
- Calcula e guarda o valor e peso da mochila para o indivíduo, associando o modo de seleção dos itens em 0 e 1.
- É gerada uma lista em ordem alfabética com os itens.
- Se o fitness encontrado para a população fosse maior que o anterior, e o peso não ultrapassasse o limite, encontra-se o melhor fitness e o melhor indivíduo. 
- Após repetir para o número máximo de gerações, e encontrarmos um resultado a cada nova melhor opção, temos o Hall da Fama.

Após executar o código algumas vezes, percebe-se que esse algoritmo é probabilístico, pois obtemos novos resultados a cada compilação. E, como conversado em aula, temos um problema do tipo NP difícil, ou seja, não há um algoritmo com 100% de eficácia, sem que testem todas as possibilidades.

Lembrando que, a função objetivo utiliza da ferramentas de penalização para reduzir o fitness de casos inviáveis, por exemplo, supodo que todos os itens ultrapassem o peso máximo da mochila individualmente, a penalização neste caso assumi o valor 0.01 e nenhum item é colocado na mochila.

## Playground

